<a href="https://colab.research.google.com/github/Yuuu553/-/blob/main/%E6%83%85%E7%B7%92%E6%97%A5%E8%A8%98ver3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#最新版本（情緒分析＋存檔）

# 安裝所需模組
!pip install gradio snownlp

import gradio as gr
import datetime
from snownlp import SnowNLP
import random  # 用來隨機選擇建議
import json
from google.colab import drive

# 掛載 Google Drive
drive.mount('/content/drive')

# 讀取 Google Drive 上的紀錄檔案
file_path = '/content/drive/MyDrive/emotion_records.json'

# 嘗試從 Google Drive 讀取現有紀錄，如果檔案不存在，則初始化空的紀錄列表
try:
    with open(file_path, 'r', encoding='utf-8') as f:
        records = json.load(f)
except FileNotFoundError:
    records = []


# 情緒建議選項
advice_options = {
    "positive": [
        "今天的情緒真是太棒了，繼續保持這份正能量，做些讓自己更快樂的事，或許可以計劃一些冒險的活動。",
        "今天你感覺非常開心！這是最好的狀態，繼續做一些能讓你快樂的事，與朋友分享這份喜悅吧！"
    ],
    "neutral": [
        "你的情緒正在走向正向，保持這份輕鬆與愉快。試著多做一些運動，或許能讓這份正能量持久。",
        "看來你今天心情不錯！可以再做些能讓你笑出來的事情，或者與家人或朋友聚會。"
    ],
    "negative": [
        "今天的情緒處於平穩狀態，可以考慮做些放鬆的事，像是散步、冥想或閱讀。",
        "心情還不錯，或許可以通過做些簡單的運動或看一些喜劇來提升你的情緒。"
    ],
    "very_negative": [
        "你的情緒有點低落，試著從小事開始改善，聽點音樂或與朋友談談心，這能幦幫助放鬆心情。",
        "感覺有些疲憊或困擾嗎？試著給自己一些時間休息，無論是喝杯茶，還是做些輕鬆的活動。"
    ],
    "extremely_negative": [
        "今天感覺很沉重吧？也許是時候停下來，深呼吸並反思，與信任的朋友或家人談談可能會有所幫助。",
        "當你感到情緒低落時，休息是最好的方法。或許可以嘗試冥想，放下壓力，給自己一點時間來恢復。"
    ]
}

# 根據情緒得分提供建議
def get_random_advice(sentiment_score):
    if sentiment_score >= 0.8:
        return random.choice(advice_options["positive"])
    elif sentiment_score >= 0.6:
        return random.choice(advice_options["positive"])
    elif sentiment_score >= 0.4:
        return random.choice(advice_options["neutral"])
    elif sentiment_score >= 0.2:
        return random.choice(advice_options["negative"])
    else:
        return random.choice(advice_options["very_negative"])

def analyze_emotion(note):
    """
    使用 SnowNLP 進行情緒分析，回傳情緒分數（0~1）。
    越接近 1 表示越正面，越接近 0 表示越負面。
    """
    s = SnowNLP(note)
    return s.sentiments

def save_record(emotion, note):
    if not emotion or not note.strip():
        return "請選擇情緒並填寫感受！", []

    # 進行情緒分析
    sentiment_score = analyze_emotion(note)

    # 根據情緒得分提供隨機建議
    advice = get_random_advice(sentiment_score)

    record = {
        "emotion": emotion,
        "note": note.strip(),
        "sentiment": round(sentiment_score, 2),  # 保留兩位小數
        "advice": advice,
        "time": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    }
    records.append(record)

    # 儲存到 Google Drive 中
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(records, f, ensure_ascii=False, indent=4)

    # 回傳訊息和更新的記錄
    return f"記錄已儲存！ 情緒: {emotion}，情緒得分: {record['sentiment']}，建議: {advice}", [
        [r["emotion"], r["note"], r["sentiment"], r["advice"], r["time"]] for r in records
    ]

def clear_records():
    global records
    records = []
    # 清除 Google Drive 中的紀錄檔案
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(records, f, ensure_ascii=False, indent=4)
    return "所有記錄已清除！", []

# Gradio 界面
with gr.Blocks() as diary_app:
    gr.Markdown("# 情緒紀錄日記")

    with gr.Row():
        with gr.Column():
            gr.Markdown("### 記錄你的情緒")
            emotion_buttons = gr.Radio(
                ["喜", "怒", "哀", "樂"], label="選擇情緒", interactive=True
            )
            note_input = gr.Textbox(
                label="寫下你的感受...", lines=4, placeholder="請輸入文字"
            )
            save_button = gr.Button("儲存記錄")

        with gr.Column():
            gr.Markdown("### 你的情緒記錄")
            history_display = gr.Dataframe(
                headers=["情緒", "感受", "情緒得分", "情緒建議", "時間"], interactive=False
            )
            clear_button = gr.Button("清除所有記錄")

    save_button.click(
        save_record,
        inputs=[emotion_buttons, note_input],
        outputs=[gr.Text(label="訊息"), history_display],
    )
    clear_button.click(
        clear_records,
        inputs=None,
        outputs=[gr.Text(label="訊息"), history_display],
    )

# 啟動應用程式
diary_app.launch()
